In [1]:
import dask
from distributed import LocalCluster, Client, performance_report, get_task_stream, get_task_metadata

In [2]:
cluster = LocalCluster()
c = Client(cluster)

In [4]:
from distributed.diagnostics.plugin import SchedulerPlugin

class MyPlugin(SchedulerPlugin):
    def __init__(self, scheduler):
        self.scheduler = scheduler

    def transition(self, key, start, finish, *args, **kwargs):        
        if start == 'processing': # and finish == 'memory':
            print(start, finish, key) # , args, kwargs)
            # Get full TaskState
            ts = self.scheduler.tasks[key]
            ts.metadata['gid'] = 'mygid'
            # print(ts.metadata['gid'])
            # print(ts.get('worker'))

In [32]:
myplugin = MyPlugin(cluster.scheduler)
cluster.scheduler.add_plugin(myplugin, adam='yay!')

# do computation
with get_task_stream() as ts:
    with get_task_metadata() as tasks:
        def inc(x):
            return x + 1

        def double(x):
            return x * 2

        def add(x, y):
            return x + y

        data = [1, 2, 3, 4, 5]

        output = []
        for x in data:
            a = dask.delayed(inc)(x, dask_key_name='my-inc-somehash')
            b = dask.delayed(double)(x)
            c = dask.delayed(add)(a, b)
            output.append(c)
        
        total = dask.delayed(sum)(output, dask_key_name='hi2')
        r = dask.compute(total)[0]#, output, a, b, c)[0]

# end
cluster.scheduler.remove_plugin(myplugin)

processing memory double-0b93e53a-3bc8-4c0d-911d-2ed607414547
processing memory double-0b93e53a-3bc8-4c0d-911d-2ed607414547
processing memory double-0b93e53a-3bc8-4c0d-911d-2ed607414547
processing memory my-inc-somehash
processing memory my-inc-somehash
processing memory my-inc-somehash
processing memory double-9fa59b7d-ab37-4598-ad3f-da96c394683d
processing memory double-9fa59b7d-ab37-4598-ad3f-da96c394683d
processing memory double-9fa59b7d-ab37-4598-ad3f-da96c394683d
processing memory double-4197422b-75b7-46c5-bed9-35d05a347199
processing memory double-4197422b-75b7-46c5-bed9-35d05a347199
processing memory double-4197422b-75b7-46c5-bed9-35d05a347199
processing memory double-b3846b1f-7fa1-4004-b430-b100cb3aaa3e
processing memory double-b3846b1f-7fa1-4004-b430-b100cb3aaa3e
processing memory double-b3846b1f-7fa1-4004-b430-b100cb3aaa3e
processing memory double-f8ed6c12-4273-43fb-b340-16578f33c872
processing memory double-f8ed6c12-4273-43fb-b340-16578f33c872
processing memory double-f8ed6

In [31]:
total.__dask_graph__()

layer_type,MaterializedLayer
is_materialized,True
layer_type,MaterializedLayer
is_materialized,True
layer_type,MaterializedLayer
is_materialized,True
layer_type,MaterializedLayer
is_materialized,True
layer_type,MaterializedLayer
is_materialized,True
layer_type,MaterializedLayer


In [14]:
tasks.metadata

{'sum-00ac8317-23b8-40a7-915e-08a44a06de8a': {'gid': 'mygid'}}